In [ ]:
!pip install -U fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

In [ ]:
# create dataset
dataset_dir = "dataset"

dataset = fo.Dataset.from_dir(
    dataset_dir = dataset_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path="data.yaml",
    overwrite=True,
    name="fix"
)
session = fo.launch_app(dataset)

In [ ]:
# load samples
for sample in dataset:
    # Load our annotation file into a list of detections
    sample_root = sample.filepath.split("/")[-1].split(".")[0]
    sample_ann_path = ann_path + "/" + sample_root + ".txt"

    with open(sample_ann_path, 'r') as file:
        list_of_anns = [line.strip().split() for line in file]

    # For each detection, adjust the format and add to our detections list
    detections = []

    for ann in list_of_anns:

        # Make sure to make adjustments to custom formats!
        # Move label up one
        label = str(int(ann[0]) + 1)

        # Adjust bounding box from x_center, y_center, length, width to top_left_x, top_left_y, width, height
        bbox = [float(x) for x in ann[1:]] # x,y,l,w
        bbox_adjusted = [bbox[0]-bbox[3]/2, bbox[1]-bbox[2]/2, bbox[3], bbox[2]] # x,y,w,h

        # Add the object to the sample
        det = fo.Detection(
            label=label, bounding_box=bbox_adjusted
        )

        detections.append(det)

    sample["ground_truth"] = fo.Detections(detections=detections)
    sample.save()

In [ ]:
# launch
dataset = fo.load_dataset("dataset")
print(dataset)
print(dataset.head())
session = fo.launch_app(dataset)
session.wait()